<a href="https://colab.research.google.com/github/EmilijusPocius/DLWTF/blob/main/TNLP_RecipeGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

#IMPORTAI

In [ ]:
!pip install -q -U google-generativeai
!pip install -q diffusers transformers accelerate

import pandas
import numpy as np
import re
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from google.colab import files
import kagglehub
import math
import os
import gdown
from tqdm.auto import tqdm
tqdm.pandas()

import google.generativeai as genai
from google.colab import userdata
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import gc

#DUOMENŲ IR MODELIO ATSISIUNTIMAS

In [ ]:
print("Atsisiunčiamas RecipeNLG duomenų rinkinys...")
path = kagglehub.dataset_download("paultimothymooney/recipenlg")
csv_path = os.path.join(path, "RecipeNLG_dataset.csv")
print(f"Duomenų path: {csv_path}")

In [ ]:
file_id = '1GVRMOc4rObbTiu7tU6FdjVCgQzX-3V5q'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'recipe_model_transformer_version_finalest_final.keras'

if not os.path.exists(output):
    print(f"Atsisiunčiamas modelis...")
    gdown.download(url, output, quiet=False)
else:
    print("Modelio failas jau yra.")

#PARAMETRAI

In [ ]:
VOCAB_SIZE = 15000
TOKENIZER_SAMPLE_SIZE = 300000
MAX_ENCODER_LEN = 25
MAX_DECODER_LEN = 100
BATCH_SIZE = 256
EMBED_DIM = 128
NUM_HEADS = 4
FF_DIM = 512
DROPOUT_RATE = 0.1
EPOCHS = 15
DATA_LIMIT = 300000

# TOKENIZARTORIAI

In [ ]:
# Tokenizatorius
print(f"Skaitoma {TOKENIZER_SAMPLE_SIZE} įrašų imtis tokenizatoriaus apmokymui...")
df_sample = pandas.read_csv(csv_path, nrows=TOKENIZER_SAMPLE_SIZE)

# Paruošiame tekstą
df_sample = df_sample.dropna(subset=['NER', 'directions'])
df_sample['input_text'] = df_sample['NER'].apply(lambda x: ' '.join(eval(x)))
df_sample['target_text'] = df_sample['directions'].apply(lambda x: '<start> ' + ' '.join(eval(x)) + ' <end>')

# Encoder Tokenizer
print("Apmokomas Encoder Tokenizer...")
encoder_tokenizer = keras.preprocessing.text.Tokenizer(num_words=VOCAB_SIZE, oov_token="<unk>")
encoder_tokenizer.fit_on_texts(df_sample['input_text'])

# Decoder Tokenizer
print("Apmokomas Decoder Tokenizer...")
decoder_tokenizer = keras.preprocessing.text.Tokenizer(
    num_words=VOCAB_SIZE,
    oov_token="<unk>",
    filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
)
decoder_tokenizer.fit_on_texts(df_sample['target_text'])

del df_sample

print("Tokenizatoriai paruošti.")

#TRENIRAVIMO DUOMENŲ GENERAVIMAS

In [ ]:
class RecipeDataGenerator(keras.utils.Sequence):
    def __init__(self, df, batch_size, encoder_tokenizer, decoder_tokenizer,
                 max_encoder_len, max_decoder_len, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.df = df
        self.batch_size = batch_size
        self.encoder_tokenizer = encoder_tokenizer
        self.decoder_tokenizer = decoder_tokenizer
        self.max_encoder_len = max_encoder_len
        self.max_decoder_len = max_decoder_len
        self.shuffle = shuffle
        self.indexes = self.df.index.tolist()
        self.on_epoch_end()

    def __len__(self):
        # Nurodome kiek batchų vienoje epochoje
        return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        # Generuojame vieną batchą duomenų
        start_index = index * self.batch_size
        end_index = (index + 1) * self.batch_size
        batch_indexes = self.indexes[start_index:end_index]

        # Paimame duomenis pagal indeksus
        df_batch = self.df.loc[batch_indexes]

        # Paruošiame duomenis
        encoder_input = self._prepare_encoder_input(df_batch['input_text'].tolist())
        decoder_input, decoder_output = self._prepare_decoder_io(df_batch['target_text'].tolist())

        return (encoder_input, decoder_input), decoder_output

    def on_epoch_end(self):
        # Sumaišome indeksus po kiekvienos epochos
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def _prepare_encoder_input(self, texts):
        return keras.preprocessing.sequence.pad_sequences(
            self.encoder_tokenizer.texts_to_sequences(texts),
            maxlen=self.max_encoder_len, padding='post'
        )

    def _prepare_decoder_io(self, texts):
        decoder_input_seq = keras.preprocessing.sequence.pad_sequences(
            self.decoder_tokenizer.texts_to_sequences(texts),
            maxlen=self.max_decoder_len, padding='post'
        )

        decoder_output_seq = np.zeros_like(decoder_input_seq)
        decoder_output_seq[:, :-1] = decoder_input_seq[:, 1:]

        return decoder_input_seq, decoder_output_seq

print("RecipeDataGenerator klasė sukurta.")

#DUOMENŲ PARUOŠIMAS (PALEISTI JEI BUS TRENIRUOJAMAS MODELIS)

In [ ]:
# print("Įkeliamas duomenų rinkinys")
# df_full = pandas.read_csv(csv_path, usecols=['NER', 'directions'])
# df_full = df_full.dropna().reset_index(drop=True)
# df_full = df_full.iloc[:DATA_LIMIT]

# print("Ruošiamas tekstas...")
# df_full['input_text'] = df_full['NER'].progress_apply(lambda x: ' '.join(eval(x)))
# df_full['target_text'] = df_full['directions'].progress_apply(lambda x: '<start> ' + ' '.join(eval(x)) + ' <end>')

# df_full.drop(columns=['NER', 'directions'], inplace=True)

# print("Duomenys dalijami į treniravimo ir validacijos aibes...")
# train_df, val_df = train_test_split(df_full, test_size=0.1, random_state=42)

# del df_full

# print(f"Treniravimo aibės dydis: {len(train_df)}")
# print(f"Validacijos aibės dydis: {len(val_df)}")

In [ ]:
# train_generator = RecipeDataGenerator(
#     df=train_df,
#     batch_size=BATCH_SIZE,
#     encoder_tokenizer=encoder_tokenizer,
#     decoder_tokenizer=decoder_tokenizer,
#     max_encoder_len=MAX_ENCODER_LEN,
#     max_decoder_len=MAX_DECODER_LEN
# )

# val_generator = RecipeDataGenerator(
#     df=val_df,
#     batch_size=BATCH_SIZE,
#     encoder_tokenizer=encoder_tokenizer,
#     decoder_tokenizer=decoder_tokenizer,
#     max_encoder_len=MAX_ENCODER_LEN,
#     max_decoder_len=MAX_DECODER_LEN,
#     shuffle=False
# )

# print("Duomenų generatoriai paruošti.")

#CUSTOM TRANSFORMERIS

In [ ]:
class PositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim, mask_zero=True
        )
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.supports_masking = True

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return self.token_embeddings.compute_mask(inputs, mask=mask)

class TransformerEncoder(keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            keras.layers.Dense(dense_dim, activation="relu"),
            keras.layers.Dense(embed_dim),
        ])
        self.layernorm_1 = keras.layers.LayerNormalization()
        self.layernorm_2 = keras.layers.LayerNormalization()
        self.dropout = keras.layers.Dropout(DROPOUT_RATE)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = tf.cast(mask, dtype="int32")
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)

        proj_output = self.dense_proj(proj_input)
        proj_output = self.dropout(proj_output)
        return self.layernorm_2(proj_input + proj_output)

class TransformerDecoder(keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            keras.layers.Dense(dense_dim, activation="relu"),
            keras.layers.Dense(embed_dim),
        ])
        self.layernorm_1 = keras.layers.LayerNormalization()
        self.layernorm_2 = keras.layers.LayerNormalization()
        self.layernorm_3 = keras.layers.LayerNormalization()
        self.dropout = keras.layers.Dropout(DROPOUT_RATE)
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is not None:
            mask = tf.cast(mask, dtype="int32")
            padding_mask = mask[:, tf.newaxis, :]
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1, value=encoder_outputs, key=encoder_outputs, attention_mask=padding_mask
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        proj_output = self.dropout(proj_output)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)


#MODELIO SUKŪRIMAS

In [ ]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(MAX_ENCODER_LEN, VOCAB_SIZE, EMBED_DIM)(encoder_inputs)
encoder_outputs = TransformerEncoder(EMBED_DIM, FF_DIM, NUM_HEADS)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
x_dec = PositionalEmbedding(MAX_DECODER_LEN, VOCAB_SIZE, EMBED_DIM)(decoder_inputs)
x_dec = TransformerDecoder(EMBED_DIM, FF_DIM, NUM_HEADS)(x_dec, encoder_outputs)

decoder_outputs = keras.layers.Dense(VOCAB_SIZE, activation="softmax")(x_dec)

transformer_model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="transformer_recipe_generator")

transformer_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

transformer_model.summary()

#MODELIO UŽKROVIMAS

In [ ]:
model_path = 'recipe_model_transformer_version_finalest_final.keras'

print(f"Loading weights into model from: {model_path}")

loaded_model = transformer_model
loaded_model.load_weights(model_path)
loaded_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

loaded_model.summary()

#MODELIO TRENIRAVIMAS (PALEISTI JEI REIKIA)

In [ ]:
# early_stopping = keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=2, restore_best_weights=True
# )
# checkpoint = keras.callbacks.ModelCheckpoint(
#     'transformer_recipe_model.keras', save_best_only=True
# )

# history = transformer_model.fit(
#     train_generator,
#     validation_data=val_generator,
#     epochs=EPOCHS,
#     callbacks=[early_stopping, checkpoint]
# )

#GENERAVIMO PARUOŠIMAS

In [ ]:
# API KEY
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    print("Failed to fetch API KEY.")

genai.configure(api_key=GOOGLE_API_KEY)
text_model_name = 'gemini-2.0-flash'
text_model = genai.GenerativeModel(text_model_name)

image_model_name = "runwayml/stable-diffusion-v1-5"

# Check for GPU availability
if torch.cuda.is_available():
    print("GPU is available. Using torch.float16 and enabling model CPU offload.")
    pipe = StableDiffusionPipeline.from_pretrained(
        image_model_name,
        torch_dtype=torch.float16
    )
    pipe.enable_model_cpu_offload()
else:
    print("GPU is NOT available. Using torch.float32 for CPU execution. Image generation will be very slow.")
    pipe = StableDiffusionPipeline.from_pretrained(
        image_model_name,
        torch_dtype=torch.float32 # Use float32 for CPU
    )

pipe.enable_attention_slicing()

print("Modeliai paruošti.")

In [ ]:
def generate_recipe_transformer(input_text, max_len=100, temperature=0.7):
    # Paruošiame ingredientus (Encoder input)
    input_seq = encoder_tokenizer.texts_to_sequences([input_text])
    input_seq = keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=MAX_ENCODER_LEN, padding='post')
    input_tensor = tf.convert_to_tensor(input_seq)

    # Pradedame instrukciją (Decoder input)
    start_token = decoder_tokenizer.word_index['<start>']
    end_token = decoder_tokenizer.word_index['<end>']

    output_seq = [start_token]

    for _ in range(max_len):
        target_seq = tf.convert_to_tensor([output_seq])

        predictions = transformer_model.predict([input_tensor, target_seq], verbose=0)

        predictions = predictions[0, -1, :]

        # Temperature Sampling
        predictions = np.log(predictions + 1e-10) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)

        # Išrenkame kitą žodį
        predicted_id = np.random.choice(len(predictions), p=predictions)

        if predicted_id == end_token:
            break

        output_seq.append(predicted_id)

    # Konvertuojame skaičius atgal į tekstą
    recipe_text = decoder_tokenizer.sequences_to_texts([output_seq])[0]

    # Išvalome <start>
    recipe_text = recipe_text.replace('<start>', '').strip()

    return recipe_text

In [ ]:
def enhance_recipe(original_ingredients, raw_instructions):
    prompt = f"""
    You are a professional chef editor. Refine this raw AI-generated recipe.

    1. CREATE A NAME: Give it a creative, appetizing name.
    2. INGREDIENTS: The input list has no quantities. Add logical quantities (e.g., '200g pasta' instead of just 'pasta').
       - If essential items (salt, oil, water, pepper) are missing, add them and mark as "(Suggested)".
    3. INSTRUCTIONS: Rewrite the raw instructions to be clear, numbered steps. Fix grammar. Do NOT change the dish, just the clarity.

    Input Ingredients: {original_ingredients}
    Raw AI Instructions: {raw_instructions}

    Output Format (Markdown):
    # [Name]
    **Description:** [One sentence summary]
    ## Ingredients
    * ...
    ## Instructions
    1. ...
    """

    safety_settings = [
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE"
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE"
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE"
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE"
        },
    ]

    try:
        # Siunčiame užklausą su saugumo nustatymais
        response = text_model.generate_content(prompt, safety_settings=safety_settings)

        # Patikriname, ar gavome atsakymą
        if response.text:
            return response.text
        else:
            return "Modelis sugeneravo tuščią atsakymą (galbūt dėl turinio filtro)."

    except Exception as e:
        # Jei kyla klaida, parodome ją ekrane vietoje recepto
        return f"**Klaida apdorojant receptą:** {str(e)}\n\nPatikrinkite API raktą arba 'Raw Output'."


In [ ]:
def generate_image(ingredients, recipe_text):
    # Ištraukiame pavadinimą paveikslėliui
    match = re.search(r'# (.*?)\n', recipe_text)
    if match:
        recipe_title = match.group(1).strip()
    else:
        recipe_title = "Delicious food dish" # Default

    image_prompt = (
        f"professional food photography of {recipe_title}, "
        f"containing {ingredients}, "
        "studio lighting, delicious, 4k, high resolution, michelin star presentation"
    )

    image = pipe(image_prompt).images[0]

    return image

In [ ]:
def generate_recipes(ingredients, temperature=0.7, count=1, generate_img=False):
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    results = []

    for i in range(count):
        raw_text = generate_recipe_transformer(ingredients, temperature=temperature)
        refined_recipe = enhance_recipe(ingredients, raw_text)

        image = None
        if generate_img:
            try:
                if "Nepavyko" in refined_recipe:
                     image_prompt_text = f"delicious {ingredients} dish, professional food photography"
                else:
                     image_prompt_text = refined_recipe
                image = generate_image(ingredients, image_prompt_text)
            except Exception as e:
                print(f"Nuotraukos generavimo klaida: {e}")

        results.append({
            "raw": raw_text,
            "refined": refined_recipe,
            "image": image
        })

    return results

#RECEPTO GENERAVIMAS

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# -- UI Elementai --
ingredients_input = widgets.Text(
    value='beef tomato potato pasta spices',
    placeholder='Įveskite ingredientus...',
    description='Ingredientai:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'}
)

temp_slider = widgets.FloatSlider(
    value=0.8,
    min=0.1,
    max=1.5,
    step=0.1,
    description='Kūrybiškumas:',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'}
)

count_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=3,
    step=1,
    description='Receptų kiekis:',
    readout=True,
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'}
)

# Checkbox nuotraukoms
image_checkbox = widgets.Checkbox(
    value=False,
    description='Generuoti nuotrauką?',
    disabled=False,
    indent=False,
    layout=widgets.Layout(width='600px')
)

run_button = widgets.Button(
    description='Generuoti',
    button_style='success',
    layout=widgets.Layout(width='200px', margin='10px 0 0 0'),
    icon='check'
)

out = widgets.Output()

def on_button_click(b):
    run_button.disabled = True
    run_button.description = "Dirbama..."

    with out:
        clear_output(wait=True)

        # Paimame reikšmes
        ing = ingredients_input.value
        temp = temp_slider.value
        cnt = count_slider.value
        do_img = image_checkbox.value

        status_msg = "⏳ Generatorius cookina"
        if do_img:
            status_msg += " ir fotografuoja patiekalą."

        print(status_msg)

        try:
            results_list = generate_recipes(ing, temperature=temp, count=cnt, generate_img=do_img)

            clear_output(wait=True)

            if not results_list:
                print("Klaida: Nieko nesugeneruota.")

            for idx, res in enumerate(results_list):
                display(Markdown(f"---"))
                display(Markdown(f"## 🍽️ Receptas #{idx + 1}"))

                # 1. Nuotrauka
                if res.get('image'):
                    display(res['image'])
                elif do_img:
                    print("Nepavyko sugeneruoti nuotrauką.")

                # 2. Išmanus (Refined) Receptas
                refined_text = str(res.get('refined', ''))

                if refined_text and len(refined_text) > 10:
                    try:
                        display(Markdown(refined_text))
                    except:
                        print(refined_text)
                else:
                    print("Nepavyko pagerinti recepto.")

                # 3. Žalias (Raw) tekstas
                raw_text = str(res.get('raw', ''))
                if raw_text:
                    display(Markdown(f"---"))
                    display(Markdown(f"**Transformerio sugeneruotas tekstas (Raw):**"))
                    display(Markdown(f"> *{raw_text}*"))

        except Exception as e:
            print(f"KLAIDA: {e}")
            import traceback
            traceback.print_exc()

        finally:
            run_button.disabled = False
            run_button.description = "Generuoti"

run_button.on_click(on_button_click)

# UI Išdėstymas
ui = widgets.VBox([
    widgets.HTML("<h2>PotatoGPT 🥔</h2>"),
    ingredients_input,
    temp_slider,
    count_slider,
    image_checkbox,
    run_button,
    out
], layout=widgets.Layout(align_items='center', width='100%'))

display(ui)